#  The Shared Library with GCC

**C/C++ Under Linux**

* %cd Change the current working directory to `./demo`

In [1]:
%cd demo

F:\SEU\SEE\PySEE\home\notebook\demo


In [2]:
%pwd

'F:\\SEU\\SEE\\PySEE\\home\\notebook\\demo'

When your program is linked against a shared library, only a small table is created in the executable. Before the executable starts running, **the operating system loads the machine code needed for the external functions** - a process known as **dynamic linking.** 

   
 ![dynamic](./img/dynamic-lib.png)  

    
* Dynamic linking makes executable files smaller and saves disk space, because `one` copy of a **library** can be **shared** between `multiple` programs. 


* Furthermore, most operating systems allows one copy of a shared library in memory to be used by all running programs, thus, saving memory. 


* The shared library codes can be upgraded without the need to recompile your program.


A **shared library** has file extension of 

   * **`.so`** (shared objects) in `Linux(Unixes)`
   
   
   * **`.dll** (dynamic link library) in `Windows`. 
   



## 1: Building the shared library

The shared library we will build consist of a single source file: 

* `statistics.c/h`

We will compile the C file with `Position Independent Code( PIC )` into a shared library。

GCC assumes that all libraries 
   
* `start` with **lib**

* `end`  with `.dll`(windows) or `.so`(Linux)，

so, we should name the  shared library begin with `lib prefix` and the `.so/.dll` extensions.

* libstat.dll(Windows)

* libstat.so(Linux)


In [2]:
%%file ./include/statistics.h

#ifndef STATISTICS_H
#define STATISTICS_H

double  mean(double data[], int size);

#endif


Overwriting ./include/statistics.h


In [3]:
%%file ./src/statistics.c

#include "statistics.h"

double  mean(double data[], int size)
{
  /* Compute the arithmetic mean of a dataset using the recurrence relation 
     mean_(n) = mean(n-1) + (data[n] - mean(n-1))/(n+1)   */

  double mean = 0;
  for(int  i = 0; i < size; i++)
  {
      mean += (data[i] - mean) / (i + 1);
   }
  return mean;
}


Overwriting ./src/statistics.c


### 1.1 Building the Lib

**Makefile of Building the lib**

In [2]:
%%file ./makefile-statlib.mk

CC=gcc
CFLAGS=-O3 -Wall -fPIC 

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR=./include/

# Linux
#LIB=libstat.so
# Windows
LIB=libstat.dll

all:  $(LIB)

$(LIB): obj
	 $(CC) -shared -o $(BINDIR)$@ $(OBJDIR)statistics.o
    
obj: $(SRCDIR)statistics.c
	 $(CC) -c $(CFLAGS)  -o $(OBJDIR)statistics.o $(SRCDIR)statistics.c -I$(INCDIR)
     


Overwriting ./makefile-statlib.mk


In [3]:
!make -f makefile-statlib.mk

gcc -shared -o ./bin/libstat.dll ./obj/statistics.o


In [4]:
!dir .\bin\libstat.dll

 Volume in drive F is cmh
 Volume Serial Number is 9C25-3306

 Directory of F:\SEU\SEE\PySEE\home\notebook\demo\bin

2020/05/14  14:49           309,891 libstat.dll
               1 File(s)        309,891 bytes
               0 Dir(s)  89,984,061,440 bytes free


#### Linux 

* LIB=libstat.so

In [ ]:
!ls ./bin/libstat.*

Linux Coommand: list files
```bash
$ls
```

##### Path separator

* Path separators

   * Linux: `/`

   * Windows: **\**

* Eescape character `\`

   * The backslash`\` is an `escape character`,you use to inform that the next character is special.
 

#### GCC Options

* `-c`: compile into object file with default name : *.o.

      By default, the object file has the same name as the source file with extension of ".o" 
  
  
* `-O3`: Optimize yet more.

      turns on all optimizations specified by -O2 and also turns on the -finline-functions, -fweb, -frename-registers and -funswitch-loops optionsturns  on  all  optimizations   
     
  
* `-Wall`: prints "all"  compiler's warning message. 

      This option should always be used, in order to generate better code.


* **`-fPIC`** : stands for `Position Independent Code`(位置无关代码)
   
   the generated machine code is `not dependent` on being located at a `specific address` in order to `work`.
   
   Position-independent code can be `executed` at `any memory address`
    
     
* **-shared:** creating a shared library


## 2 Building a Client Executable 

### 2.1 Header Files and Libraries 

* `Header File`: When compiling the program, the **compiler** needs the **header** files to compile the source codes;

* `libraries`: the **linker** needs the **libraries** to resolve external references from other object files or libraries. 

The `compiler` and `linker` will not find the `headers/libraries` unless you set **the appropriate options**

* **1 Searching for Header Files**

   **`-Idir`:** The include-paths are specified via **-Idir** option (`uppercase` 'I' followed by the directory path or environment variable **CPATH**). 
   
   
* **2 Searching for libraries Files**

   **`-Ldir`**: The library-path is specified via **-Ldir** option (`uppercase` 'L' followed by the directory path(or environment variable **LIBRARY_PATH**). 


* **3 Linking the library**

   **`-llibname`**: Link with the library name **without** the `lib` prefix and the `.dll/.so` extensions.
  
   Windows
   ```bash
      -I./include/ 
      -L./bin/ -lstat
  ```
>**Linux**
> ```bash
>   -I./include/ 
    -L./bin/ -lstat -Wl,-rpath=./bin/  
> ```
>* **`-Wl,option`**
>
>    Pass option as an option to the **linker**. 
>    
>    If option contains `commas`, it is split into multiple options at the commas. 
>    
>    You can use this syntax to pass an argument to the option.
>      
>   
>* **`-rpath=dir`** 
>
>   **Add a directory to the runtime library search path**. This is used when linking an >executable with shared objects. All -rpath arguments are concatenated and passed to the >runtime linker, which uses them to locate shared objects at runtime. 
   


### 2.2 Calling the Shared Library

The following source code demonstrates calling the shared library's functions: 

**NOTE:** the code is the **same** code in multi-source example

In [5]:
%%file ./src/statApp.c

#include <stdio.h> 
#include "statistics.h"

int main() {
     double a[] = {8, 4, 5, 3, 2};
     int length = sizeof(a)/sizeof(double);  
     printf("mean is %f\n", mean(a,length)); 
     return 0;
}


Overwriting ./src/statApp.c


**Makefile**

In [6]:
%%file ./makefile-statapp.mk

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR=./include/

all: statApp

statApp: statobj  
	gcc -o $(BINDIR)statApp $(OBJDIR)statApp.o -L$(BINDIR) -lstat
#	gcc -o $(BINDIR)statApp $(OBJDIR)statApp.o -L$(BINDIR) -lstat -Wl,-rpath=./bin/  

statobj: $(SRCDIR)statApp.c 
	gcc -c -o $(OBJDIR)statApp.o $(SRCDIR)statApp.c -I$(INCDIR)

Overwriting ./makefile-statapp.mk


In [7]:
!make -f makefile-statapp.mk

gcc -c -o ./obj/statApp.o ./src/statApp.c -I./include/
gcc -o ./bin/statApp ./obj/statApp.o -L./bin/ -lstat


#### Windows

In [8]:
!.\bin\statApp

mean is 4.400000


**Linux**

In [ ]:
!./bin/statApp

## 3 Building  a Shared Library with Multi-source

### 3.1  The multi-source files
The shared library we will build consist of the multi-source files

* statistics.c/h
* funs.c/h

**funs.c/h N!**

In [10]:
%%file ./include/funs.h

#ifndef FUNS_H
#define FUNS_H

int factorial(int n);

#endif

Overwriting ./include/funs.h


In [11]:
%%file ./src/funs.c

#include "funs.h"

// The factorial of a positive integer n, denoted by n!, 
//    is the product of all positive integers less than or equal to n. 
//  For example,5!=5*4*3*2*1=120
//  The value of 0! is 1 
int factorial(int n)
{
    if (n == 0 ) {
        return 1;
    }
    else 
    {
        return n * factorial(n - 1);
    }
}

Overwriting ./src/funs.c


### 3.2 Building  into One Shared Library

**Makefile**

```make
SRCS=$(SRCDIR)statistics.c \
    $(SRCDIR)funs.c 
```    

In [8]:
%%file ./makefile-libfuns.mk

CC=gcc
CFLAGS=-O3 -Wall -fPIC

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR= ./include/

# Linux
# LIB=libfuns.so 
LIB=libfuns.dll

SRCS=$(SRCDIR)statistics.c \
    $(SRCDIR)funs.c 

# non-path filename
filename=$(notdir $(SRCS))

# the obj target of a source code using the pattern rule
OBJS=$(patsubst %.c,$(OBJDIR)%.o,$(filename))

all:$(LIB)
    
$(LIB): $(OBJS)  
	$(CC) -shared -o $(BINDIR)$@ $(OBJS) 

# the pattern rule: one step rule for multiple source files
$(OBJS):$(SRCS)
	$(CC) $(CFLAGS) -o $@ -c $(SRCDIR)$(patsubst  %.o,%.c,$(notdir $@))  -I$(INCDIR)

Overwriting ./makefile-libfuns.mk


In [9]:
!make -f makefile-libfuns.mk

gcc -O3 -Wall -fPIC -o obj/statistics.o -c ./src/statistics.c  -I./include/
gcc -O3 -Wall -fPIC -o obj/funs.o -c ./src/funs.c  -I./include/
gcc -shared -o ./bin/libfuns.dll ./obj/statistics.o ./obj/funs.o 


In [10]:
!dir .\bin\libf*.dll

 驱动器 F 中的卷是 cmh
 卷的序列号是 9C25-3306

 F:\SEU\SEE\PySEE\home\notebook\demo\bin 的目录

2020/06/07  14:52           310,189 libfuns.dll
               1 个文件        310,189 字节
               0 个目录 91,521,265,664 可用字节


### 3.3 Building a client executable 

The source code `"funsApp.c"` demonstrates calling the shared library's functions:


In [11]:
%%file ./src/funsApp.c

#include <stdio.h> 
#include "statistics.h"
#include "funs.h"

int main() {
    
     double a[] = {8, 4, 5, 3, 2};
     int len= sizeof(a)/sizeof(*a);
     printf("mena is %f\n", mean(a,  len)); 
    
     int n =5;
     printf("the factorial of  %d is %d\n",n,factorial(n));  // 5!=120
     return 0;
}

Overwriting ./src/funsApp.c


#### Windows 

In [12]:
!gcc -c -o ./obj/funsApp.o ./src/funsApp.c -I./include/
!gcc -o  ./bin/funsApp ./obj/funsApp.o  -L./bin/ -lfuns

In [13]:
!.\bin\funsApp

mena is 4.400000
the factorial of  5 is 120


#### Linux

In [ ]:
!gcc -c -o ./obj/funsApp.o ./src/funsApp.c -I./include/
!gcc -o  ./bin/funsApp ./obj/funsApp.o  -L./bin/ -lfuns -Wl,-rpath=./bin/ 

In [ ]:
!./bin/funsApp

## 4 CoolProp Shared Library in Windows

* [Official Doc: CoolProp Shared Library](http://www.coolprop.org/coolprop/wrappers/SharedLibrary/index.html#)

The example use a MinGW64 64-bit dll: **libCoolProp.dll** of CoolProp


* [demo call the shared library](./coolpropdemo)

![coolpropdemo](./img/vcr/coolpropdemo.jpg)

**Change working dir to `./notebook/coolpropdemo`**

In [11]:
%pwd

'J:\\SEU\\SEE\\PySEE\\home\\notebook\\coolpropdemo'

In [8]:
%cd coolpropdemo

J:\SEU\SEE\PySEE\home\notebook\coolpropdemo


In [ ]:
# %load demo.cpp
/* 
The example use a MinGW64 64-bit stdcall dll of CoolProp:

g++  -m64 -O3 -o ./bin/demo.exe  -DCOOLPROP_LIB demo.cpp -I./include -L./lib -lCoolProp
*/

#define EXPORT_CODE extern "C" __declspec(dllimport)
#define CONVENTION __stdcall
#include "CoolPropLib.h"
#undef EXPORT_CODE
#undef CONVENTION

#include <iostream>
//---------------------------------------------------------------------------
int main()
{
    std::cout <<"Demo CoolProp"<< std::endl;
    std::cout << PropsSI("T","P",101325,"Q",0,"Water") << std::endl;
    return 1;
}

编译

In [12]:
!g++  -O3 -o ./bin/demo.exe  -DCOOLPROP_LIB demo.cpp -I./include -L./lib -lCoolProp

运行

In [14]:
!.\bin\demo

Demo CoolProp
373.124
